In [1]:
# ✅ Install dependencies
!pip install pathway bokeh panel --quiet

In [2]:
import pandas as pd
import numpy as np
import pathway as pw
import datetime
import panel as pn
import bokeh.plotting
import sys
import contextlib

pn.extension()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Read dataset
df = pd.read_csv("dataset.csv")

# Combine timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
df = df.sort_values("Timestamp")

# Encode TrafficConditionNearby
def encode_traffic(x):
    return {"low": 0.0, "average": 0.5, "high": 1.0}.get(x.lower(), 0.5)

# Encode VehicleType
def encode_vehicle(x):
    return {"bike": 0.5, "car": 1.0, "truck": 1.5, "cycle": 0.2}.get(x.lower(), 1.0)

df['TrafficLevel'] = df['TrafficConditionNearby'].astype(str).apply(encode_traffic)
df['VehicleWeight'] = df['VehicleType'].astype(str).apply(encode_vehicle)

# Compute occupancy ratio
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']

# Save cleaned stream file
df[['Timestamp', 'SystemCodeNumber', 'OccupancyRatio', 'QueueLength', 'TrafficLevel', 'IsSpecialDay', 'VehicleWeight']].to_csv("model2_stream.csv", index=False)

In [ ]:
class DemandSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    OccupancyRatio: float
    QueueLength: int
    TrafficLevel: float
    IsSpecialDay: int
    VehicleWeight: float

data = pw.demo.replay_csv("model2_stream.csv", schema=DemandSchema, input_rate=100)

# Convert timestamp and create daily bucket
fmt = "%Y-%m-%d %H:%M:%S"
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
# Tuned weights
alpha = 0.1
beta = 0.05
gamma = 0.02
delta = 0.02
epsilon = 0.03
base_price = 10
lambda_scale = 1.5

windowed = (
    data.windowby(
        time_expr=pw.this.t,
        instance=pw.this.SystemCodeNumber + pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        SystemCodeNumber = pw.reducers.any(pw.this.SystemCodeNumber),
        day = pw.reducers.any(pw.this.day),
        t = pw.this._pw_window_end,
        sum_demand = pw.reducers.sum(
            alpha * pw.this.OccupancyRatio +
            beta * pw.this.QueueLength +
            gamma * pw.this.TrafficLevel +
            delta * pw.this.IsSpecialDay +
            epsilon * pw.this.VehicleWeight
        ),
        count = pw.reducers.count()
    )
)

# Demand and normalized demand
windowed = windowed.with_columns(
    demand = pw.this.sum_demand / pw.this.count,
    demand_normalized = pw.apply(lambda d: max(0, min(1, d)), pw.this.sum_demand / pw.this.count)
)

# Dynamic pricing
windowed = windowed.with_columns(
    price = pw.apply(lambda d: base_price * (1 + lambda_scale * d), pw.this.demand_normalized)
)

In [ ]:
# Get all unique parking lots
lots = pd.read_csv("model2_stream.csv")["SystemCodeNumber"].unique().tolist()

# Plot function
def plot_price_per_lot(selected_lot):
    table_df = windowed.select(
        t = pw.this.t,
        SystemCodeNumber = pw.this.SystemCodeNumber,
        price = pw.this.price
    ).filter(pw.this.SystemCodeNumber == selected_lot)

    def lot_plot(source):
        fig = bokeh.plotting.figure(
            title=f"Model 2 – Demand-Based Price for {selected_lot}",
            x_axis_type="datetime",
            height=400,
            width=800
        )
        fig.line("t", "price", source=source, line_width=2, color="green")
        fig.circle("t", "price", source=source, size=6, color="green")
        return fig

    return table_df.plot(lot_plot, sorting_col="t")

# UI dropdown + plot
dropdown = pn.widgets.Select(name="Select Parking Spot", options=lots, value=lots[0])
plot_panel = pn.bind(plot_price_per_lot, selected_lot=dropdown)

pn.Column("## Model 2 – Demand-Based Price per Lot", dropdown, plot_panel).servable()

In [ ]:
# Save output
pw.io.jsonlines.write(
    windowed.select(
        t = pw.this.t,
        SystemCodeNumber = pw.this.SystemCodeNumber,
        price = pw.this.price
    ),
    "demand_model_daily_output.jsonl"
)

# Run pipeline
with contextlib.redirect_stdout(sys.stdout), contextlib.redirect_stderr(sys.stderr):
    pw.run().await_termination()